In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
model=Sequential()

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='LHRvmbHLoIM-b2mYncTDMDI70Srcr_zKf6ElSGa4ywf-',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'aircraft-donotdelete-pr-oq3st0454ilb0h'
object_key = 'Aircraft Dent Detection.zip'

streaming_body_4 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/



In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_4.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
ls

 aircraft-classification-model.tgz  'Aircraft Dent Detection'/   Aircraft.h5


In [10]:
pwd

'/home/wsuser/work'

In [11]:
batch_size=24
train_generator = train_datagen.flow_from_directory(
    r'/home/wsuser/work/Aircraft Dent Detection/train/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 290 images belonging to 2 classes.


In [12]:
validation_generator = validation_datagen.flow_from_directory(
    r'/home/wsuser/work/Aircraft Dent Detection/test/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 21 images belonging to 2 classes.


In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size
)


Epoch 1/10
12/12 [==============================] - 84s 7s/step - loss: 1.0676 - accuracy: 0.8872
Epoch 2/10
12/12 [==============================] - 90s 8s/step - loss: 0.3181 - accuracy: 0.8383
Epoch 3/10
12/12 [==============================] - 98s 8s/step - loss: 0.2312 - accuracy: 0.9060
Epoch 4/10
12/12 [==============================] - 99s 8s/step - loss: 0.2302 - accuracy: 0.9167
Epoch 5/10
12/12 [==============================] - 98s 8s/step - loss: 0.1975 - accuracy: 0.9135
Epoch 6/10
12/12 [==============================] - 99s 8s/step - loss: 0.1641 - accuracy: 0.9410
Epoch 7/10
12/12 [==============================] - 95s 8s/step - loss: 0.1599 - accuracy: 0.9211
Epoch 8/10
 2/12 [====>.........................] - ETA: 58s - loss: 0.0854 - accuracy: 0.9792 

In [ ]:
test_loss, test_accuracy = model.evaluate(train_generator, verbose=2)
print("Test accuracy:", test_accuracy)


In [ ]:
image_path = '/home/wsuser/work/Aircraft Dent Detection/test/defect/0523005-222-Weu-0523005-100-Cessna-172L-Wing-Structure-Assy-RH-Damaged-3D_jpg.rf.0929846f483070a518d43caa58438aed.jpg'
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)
image_array = tf.expand_dims(image_array, 0)
prediction = model.predict(image_array)
if prediction[0][0] >= 0.5:
    print("Defective")
else:
    print("Non-Defective")


In [ ]:
image

In [ ]:
x=image_array
x

In [ ]:
x.ndim

In [ ]:
x=np.expand_dims(x,axis=0)
print(x)

In [ ]:
x.ndim

In [ ]:
model.save('Aircraft.h5')

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com",
                 "apikey" : "VHCRqLhOp8v7TtfudpYguvVq6Wq1coPs5CBzKPG63axX"}
client=APIClient(wml_credentials)

In [ ]:
client

In [ ]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])
    

In [ ]:
space_uid=guid_from_space_name(client,'model')
print("Space UID ="+space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.2-py3.10")
software_spec_uid

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
ls

In [ ]:
!tar -zcvf aircraft-classification-model.tgz Aircraft.h5

In [ ]:
'''model_details=client.repository.store_model(model="aircraft-classification-model.tgz",
meta_props={client.repository.ModelMetaNames.NAME:"Aircraft",
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.2",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)'''

In [ ]:
model_id=client.repository.get_model_id(model_details)

In [ ]:
model_id

In [40]:
y=train_generator[0]
y

(array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          ...,
          [0.0000000e+00, 3.9215689e-03, 0.0000000e+00],
          [0.0000000e+00, 3.9215689e-03, 0.0000000e+00],
          [0.0000000e+00, 3.9215689e-03, 0.0000000e+00]],
 
         [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          ...,
          [0.0000000e+00, 2.1840080e-03, 0.0000000e+00],
          [0.0000000e+00, 1.3521847e-03, 0.0000000e+00],
          [0.0000000e+00, 5.2036130e-04, 0.0000000e+00]],
 
         [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          ...,
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 0.0000000e+